# Cleaning and wrangling the data

<br>

## 1. Putting the data into a single dataframe

In its raw format, the data is stored in 12 csv files corresponding to each month of the analyzed year. The goal at the start is to load those files into a single pandas dataframe.

Importing the required modules.

In [1]:
import os
import pandas as pd
import numpy as np

Load each file in the raw_data directory into a combined pandas dataframe.

In [2]:
for i, file_name in enumerate(os.listdir('raw_data')):
    file_path = os.path.join('raw_data', file_name)
    file_df = pd.read_csv(file_path, index_col=None)
    
    if i == 0:  # create a combined dataframe
        combined_df = file_df
    else:       # add current file's dataframe to the combined one
        combined_df = pd.concat([combined_df, file_df], ignore_index=True)

print('n of (rows, columns):', combined_df.shape)

n of (rows, columns): (186850, 6)


Now, all of the data is is loaded into a single dataframe, combined_df.

<br>

## 2. Data quality assessment

First, look at first and last 10 entries in the data

In [3]:
combined_df.head(10)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"
6,176562,USB-C Charging Cable,1,11.95,04/29/19 13:03,"381 Wilson St, San Francisco, CA 94016"
7,176563,Bose SoundSport Headphones,1,99.99,04/02/19 07:46,"668 Center St, Seattle, WA 98101"
8,176564,USB-C Charging Cable,1,11.95,04/12/19 10:58,"790 Ridge St, Atlanta, GA 30301"
9,176565,Macbook Pro Laptop,1,1700,04/24/19 10:38,"915 Willow St, San Francisco, CA 94016"


In [4]:
combined_df.tail(10)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
186840,259349,AAA Batteries (4-pack),1,2.99,09/01/19 22:14,"911 River St, Dallas, TX 75001"
186841,259350,Google Phone,1,600,09/30/19 13:49,"519 Maple St, San Francisco, CA 94016"
186842,259350,USB-C Charging Cable,1,11.95,09/30/19 13:49,"519 Maple St, San Francisco, CA 94016"
186843,259351,Apple Airpods Headphones,1,150,09/01/19 19:43,"981 4th St, New York City, NY 10001"
186844,259352,USB-C Charging Cable,1,11.95,09/07/19 15:49,"976 Forest St, San Francisco, CA 94016"
186845,259353,AAA Batteries (4-pack),3,2.99,09/17/19 20:56,"840 Highland St, Los Angeles, CA 90001"
186846,259354,iPhone,1,700,09/01/19 16:00,"216 Dogwood St, San Francisco, CA 94016"
186847,259355,iPhone,1,700,09/23/19 07:39,"220 12th St, San Francisco, CA 94016"
186848,259356,34in Ultrawide Monitor,1,379.99,09/19/19 17:30,"511 Forest St, San Francisco, CA 94016"
186849,259357,USB-C Charging Cable,1,11.95,09/30/19 00:18,"250 Meadow St, San Francisco, CA 94016"


Next, look at some basic information about the columns.

In [5]:
combined_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


In [6]:
combined_df.describe()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
count,186305,186305,186305,186305,186305,186305
unique,178438,20,10,24,142396,140788
top,Order ID,USB-C Charging Cable,1,11.95,Order Date,Purchase Address
freq,355,21903,168552,21903,355,355


In [7]:
n_duplicated = combined_df[combined_df["Order ID"] != "Order ID"].duplicated().sum()
pct_duplicated = round(100 * n_duplicated / combined_df.shape[0], 3)
print('duplicated rows:', n_duplicated, 'rows or ', pct_duplicated, '%')

duplicated rows: 808 rows or  0.432 %


In [8]:
for col_name in combined_df.columns:
    n_null = combined_df[col_name].isna().sum()
    pct_null = round(100 * n_null / combined_df.shape[0], 3)
    print('rows with null {}: {} rows, or {} %'.format(col_name, n_null, pct_null))

rows with null Order ID: 545 rows, or 0.292 %
rows with null Product: 545 rows, or 0.292 %
rows with null Quantity Ordered: 545 rows, or 0.292 %
rows with null Price Each: 545 rows, or 0.292 %
rows with null Order Date: 545 rows, or 0.292 %
rows with null Purchase Address: 545 rows, or 0.292 %


Observations so far:
- There are 20 unique products in the dataframe,
- 355 rows contain column names instead of values,
- duplicated rows make up 0.432 % of data (not counting column names in data fields or empty rows),
- empty rows constitute 0.292 % of data,
- there are some orders with more than one type of product ordered,
- there is no actual missing data detected so far (only completely empty rows),
- all columns are of data type "string" (pandas object)

<br>

## 3. Data cleaning

First, count the number of duplicated rows (without counting the first occurance).

In [9]:
print("n of duplicated rows:", combined_df.duplicated().sum())

n of duplicated rows: 1162


Drop the duplicated rows in place and check if the number of duplicated rows is indeed 0.

In [10]:
combined_df.drop_duplicates(keep='first', inplace=True)
print("n of duplicated rows:", combined_df.duplicated().sum())

n of duplicated rows: 0


<br>

Show the count of missing values in each column.

In [11]:
combined_df.isna().sum()

Order ID            1
Product             1
Quantity Ordered    1
Price Each          1
Order Date          1
Purchase Address    1
dtype: int64

Drop rows where all values are not available and check again.

In [12]:
combined_df.dropna(how='all', inplace=True)
combined_df.isna().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
dtype: int64

<br>

Show the rows where either "Order ID" or "Quantity Ordered" column is not a digit.

In [13]:
combined_df[(combined_df['Order ID'].str.isdigit() == False) | 
            (combined_df['Quantity Ordered'].str.isdigit() == False)]

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
519,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


Replace the dataframe with a filtered out version of itself.

In [14]:
combined_df = combined_df[(combined_df['Order ID'].str.isdigit() == True) | 
                          (combined_df['Quantity Ordered'].str.isdigit() == True)]

<br>

## 4. Data wrangling

Change data types of the 'Order ID', 'Quantity Ordered' and 'Price Each' columns to int64, int16 and float respectively.

In [15]:
combined_df['Order ID'] = combined_df['Order ID'].astype('int32')
combined_df["Quantity Ordered"] = combined_df["Quantity Ordered"].astype('int16')
combined_df["Price Each"] = combined_df["Price Each"].astype('float')

<br>

Convert 'Order Date' column do datetime data type.

In [16]:
combined_df['Order Date'] = pd.to_datetime(combined_df['Order Date'])

<br> Show data types of each column

In [17]:
print(combined_df.dtypes)
combined_df.head()

Order ID                     int32
Product                     object
Quantity Ordered             int16
Price Each                 float64
Order Date          datetime64[ns]
Purchase Address            object
dtype: object


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001"
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001"
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001"


<br>

Extract the month and time fields from 'Order Date' and rearrange the columns.

In [18]:
combined_df['Order Month'] = pd.DatetimeIndex(combined_df["Order Date"]).month
combined_df["Order Time"] = pd.DatetimeIndex(combined_df["Order Date"]).time
combined_df = combined_df[["Order ID", "Product", "Quantity Ordered", "Price Each",
                           "Order Date", "Order Month", "Order Time", "Purchase Address"]]
combined_df.head(3)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Order Month,Order Time,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,4,08:46:00,"917 1st St, Dallas, TX 75001"
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,4,22:30:00,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,4,14:38:00,"669 Spruce St, Los Angeles, CA 90001"


<br> Split the address into street, city, state symbol, postcode

In [19]:
# create a dataframe from the split "Purchase Address" column and rename its columns
expanded_address_df = combined_df.loc[:, 'Purchase Address'].str.split(', ', expand=True)
expanded_address_df.rename(columns={0: "Purchase Address Street",1: "Purchase Address City",
                                    2: "Acronym & zip code"}, inplace=True)

# split the third column, containing state acronym and zip code, into two columns
expanded_address_df[["Purchase Address State", "Purchase Address Zip Code"]] = \
                                                        expanded_address_df["Acronym & zip code"].str.split(' ', expand=True)

# drop the third column
expanded_address_df.drop(columns="Acronym & zip code", inplace=True)

# concatenate the original dataframe with expanded address fields
combined_df = pd.concat([combined_df, expanded_address_df], axis=1)
combined_df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Order Month,Order Time,Purchase Address,Purchase Address Street,Purchase Address City,Purchase Address State,Purchase Address Zip Code
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,4,08:46:00,"917 1st St, Dallas, TX 75001",917 1st St,Dallas,TX,75001
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,4,22:30:00,"682 Chestnut St, Boston, MA 02215",682 Chestnut St,Boston,MA,02215
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,4,14:38:00,"669 Spruce St, Los Angeles, CA 90001",669 Spruce St,Los Angeles,CA,90001
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,4,14:38:00,"669 Spruce St, Los Angeles, CA 90001",669 Spruce St,Los Angeles,CA,90001
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,4,09:27:00,"333 8th St, Los Angeles, CA 90001",333 8th St,Los Angeles,CA,90001


<br> Create calculated "Total Price" field and reorder the columns.

In [20]:
# create a calculated field
combined_df["Total Price"] = combined_df["Quantity Ordered"] * combined_df["Price Each"]

# apply a new column order
new_column_order = ["Order ID", "Product", "Quantity Ordered", "Price Each", "Total Price", "Order Date", "Order Month",
                    "Order Time", "Purchase Address", "Purchase Address Street", "Purchase Address City",
                    "Purchase Address State", "Purchase Address Zip Code"]
combined_df = combined_df[new_column_order]

combined_df.head()

,Order ID,Product,Quantity Ordered,Price Each,Total Price,Order Date,Order Month,Order Time,Purchase Address,Purchase Address Street,Purchase Address City,Purchase Address State,Purchase Address Zip Code
0,176558,USB-C Charging Cable,2,11.95,23.90,2019-04-19 08:46:00,4,08:46:00,"917 1st St, Dallas, TX 75001",917 1st St,Dallas,TX,75001
2,176559,Bose SoundSport Headphones,1,99.99,99.99,2019-04-07 22:30:00,4,22:30:00,"682 Chestnut St, Boston, MA 02215",682 Chestnut St,Boston,MA,02215
3,176560,Google Phone,1,600.00,600.00,2019-04-12 14:38:00,4,14:38:00,"669 Spruce St, Los Angeles, CA 90001",669 Spruce St,Los Angeles,CA,90001
4,176560,Wired Headphones,1,11.99,11.99,2019-04-12 14:38:00,4,14:38:00,"669 Spruce St, Los Angeles, CA 90001",669 Spruce St,Los Angeles,CA,90001
5,176561,Wired Headphones,1,11.99,11.99,2019-04-30 09:27:00,4,09:27:00,"333 8th St, Los Angeles, CA 90001",333 8th St,Los Angeles,CA,90001


<br> Split the dataframe into two dataframes:
- first one for order items, where information about quantity and price of all order items will be stored,
- second for shipping oriented information about orders, with only one row per order.

In [21]:
order_items_cols = ["Order ID", "Product", "Quantity Ordered", "Price Each", "Total Price"]
order_items_df = combined_df.loc[:, order_items_cols]
order_items_df.head(3)

,Order ID,Product,Quantity Ordered,Price Each,Total Price
0,176558,USB-C Charging Cable,2,11.95,23.90
2,176559,Bose SoundSport Headphones,1,99.99,99.99
3,176560,Google Phone,1,600.00,600.00


When creating the second dataframe, the orders with multiple products are flagged as duplicated. Those duplicated rows are dropped, leaving the first occurance of the row.

In [22]:
# create the orders dataframe from selection of columns
orders_cols = ["Order ID", "Order Date", "Order Month", "Order Time", "Purchase Address", "Purchase Address Street",
               "Purchase Address City", "Purchase Address State", "Purchase Address Zip Code"]
orders_df = combined_df.loc[:, orders_cols]

# drop rows with duplicated Order ID column, keeping the first occurance
orders_df.drop_duplicates(subset="Order ID", keep='first', inplace=True)
orders_df.head()

,Order ID,Order Date,Order Month,Order Time,Purchase Address,Purchase Address Street,Purchase Address City,Purchase Address State,Purchase Address Zip Code
0,176558,2019-04-19 08:46:00,4,08:46:00,"917 1st St, Dallas, TX 75001",917 1st St,Dallas,TX,75001
2,176559,2019-04-07 22:30:00,4,22:30:00,"682 Chestnut St, Boston, MA 02215",682 Chestnut St,Boston,MA,02215
3,176560,2019-04-12 14:38:00,4,14:38:00,"669 Spruce St, Los Angeles, CA 90001",669 Spruce St,Los Angeles,CA,90001
5,176561,2019-04-30 09:27:00,4,09:27:00,"333 8th St, Los Angeles, CA 90001",333 8th St,Los Angeles,CA,90001
6,176562,2019-04-29 13:03:00,4,13:03:00,"381 Wilson St, San Francisco, CA 94016",381 Wilson St,San Francisco,CA,94016


<br> Show the final shapes of the cleaned and wrangled dataframes.

In [23]:
print(order_items_df.shape)
print(orders_df.shape)

(185686, 5)
(178437, 9)


<br>

## 5. Export the processed data

Save the order_items and orders dataframes to .csv files.

In [24]:
# create a directory 'clean_data' if it doesn't exist
export_dir_path = os.path.join(os.getcwd(), 'clean_data')
if not os.path.exists(export_dir_path):
    os.makedirs(export_dir_path)

order_items_df.to_csv(os.path.join(export_dir_path, 'order_items.csv'), index=False)
orders_df.to_csv(os.path.join(export_dir_path, 'orders.csv'), index=False)

<br>

Now, the cleaned and wrangled data can be found in the clean_data directory in the form of two csv files.